In [2]:
import requests
from bs4 import  BeautifulSoup as bs
import math
import re

In [4]:
rs = requests.session()
res = rs.get('http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49?fullStatus=NONE')
soup = bs(res.text)

In [5]:
count  = soup.select('.v-line')[0].text.split(' ')
print count[1]
page = int(math.ceil(float(count[1]) / 18))
print page

121
7


In [7]:
urlFormat = 'http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49/{}?&fullStatus=NONE'
with open('D:\\Big Data\\Project\\Travel\\EZtravel\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','w') as f:
    for pageNum in range(1,page+1):
        url = urlFormat.format(pageNum)
        res2 = rs.get(url)
        soup1 = bs(res2.text)
        list_table = soup1.select('div#listTable')[0]
        for items in list_table.select('div.list-box.mainLinkBox'):
            f.write('http://vacation.eztravel.com.tw/pkgfrn/' + items.select('a')[0]['href'].split('/pkgfrn/')[1] + '\n')

In [9]:
str = 'http://vacation.eztravel.com.tw/pkgfrn/introduction/FRN0000014050/20160524'
get = str.split('/')
print get

['http:', '', 'vacation.eztravel.com.tw', 'pkgfrn', 'introduction', 'FRN0000014050', '20160524']


In [10]:
dic = {} #宣告一個dic，稍後會將檔名(Key)跟網址(Value)一一放入
for bid_url in open('D:\\Big Data\\Project\\Travel\\EZtravel\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','r'):
    splitStr = bid_url.split('/')
    dataName = splitStr[5].strip() + "_" + splitStr[6].strip()
    if dataName not in dic:
        dic[dataName] = bid_url
    else:
        print dataName,dic[dataName]
#     print dataName

In [12]:
for bid_url in open('D:\\Big Data\\Project\\Travel\\EZtravel\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','r'):
    bid_detail = requests.get(bid_url.strip())
    soup2 = bs(bid_detail.text)
    print soup2.select('div.row')[0]

<div class="row">
<div class="print-date visible-print">列印日期：<span id="currentDate"></span></div>
<!-- 訂購流程圖 -->
<div class="hidden-print">
<div class="col-sm-12 hidden-ie">
<div class="step row">
<div class="bread flat">
<a class="active" href="#">行程介紹</a>
<a class="" href="#">選擇旅客人數</a>
<a class="" href="#">填寫訂購單</a>
<a class="" href="#">選擇付款方式</a>
<a class="" href="#">完成訂購</a>
</div>
</div>
</div>
<div class="col-sm-12 visible-ie">
<div class="step row"></div>
</div></div>
<div class="col-sm-12">
<div class="row">
<div class="line-block css-table">
<div class="pro-title css-td">
<h1>名名立山真好玩～漫步大雪谷(4/16~6/22).上高地.飛驒高山小京都.白川鄉合掌村.溫泉4日</h1>
<div class="gift-box">
<a class="gift-item">
<i class="icon-save"></i>
            送100元抵用金
        </a>
</div>
</div>
<div class="pro-id-left css-td">商編</div>
<div class="pro-id-right css-td">
<span> VDR0000001943  </span>
<span>NGO04IT6601G</span>
</div>
</div>
</div>
</div>
<div class="col-sm-12 product-top">
<div class="row">
<div class="col-xs-6 

KeyboardInterrupt: 

In [6]:
# with open('D:\\Big Data\\Project\\Travel\\EZtravel\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','r'):
dic = {
    'Title':'',
    'Price':'',
    'Area':'',
    'Plane':'',
    'Time':'',
    'Hotel':'',
    'Schedule':'',
    'Item ID':''
}
bid_detail = requests.get('http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001888/NGO04160602A')
soup2 = bs(bid_detail.text)
row = soup2.select('div#pkgfrnVisitHistory')[0]
price = re.search('\d+', row.select('.price')[0].text)
plane= re.search('.{4}', row.select('div#flight-block1 span')[3].text)
print row.select('div.pro-title.css-td h1')[0].text, price.group(0), row.select('.tag')[0].text,\
      row.select('div#flight-block1 span')[2].text, row.select('div#flight-block1 span')[6].text,\
      plane.group(0)
# print row.select('div#flight-block1 span')[].text

【中華航空】漫步黑部立山大雪谷.白川鄉合掌村.惠那峽.飛驒牛溫泉４日 21900 名古屋 07:30 - 11:15 09:55 - 12:20 中華航空


In [7]:
### 印出字典的key與value########
def PrintKeyValue(dic_in):
    for key, value in dic_in.iteritems() :
        print key,'  :  ', value
########################    

In [8]:
import requests
from bs4 import  BeautifulSoup as bs
# import math
import re
#最後將travel List存到此字典
bigDic={}
# 放各個行程
travel=[]
# 裝各別形成的資料
dic={}

In [9]:
bid_detail = requests.get('http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001888/NGO04160602A')
soup2 = bs(bid_detail.text, "lxml")
row = soup2.select('div#pkgfrnVisitHistory')[0]
price = re.search('\d+', row.select('.price')[0].text)
plane= re.search('.{4}', row.select('div#flight-block1 span')[3].text)

dic['title']=row.select('div.pro-title.css-td h1')[0].text
dic['price']=price.group(0)
dic['city']=row.select('.tag')[0].text     
dic['time']=row.select('div#flight-block1 span')[2].text
dic['day1']=row.select('div.day-box h4')[0].text      
dic['day2']=row.select('div.day-box h4')[1].text 
dic['day3']=row.select('div.day-box h4')[2].text
dic['day4']=row.select('div.day-box h4')[3].text
dic['planeCom']=plane.group(0)
dic['pid1']=row.select('div.pro-id-right.css-td span')[0].text.strip() 
dic['pid2']=row.select('div.pro-id-right.css-td span')[1].text.strip()


PrintKeyValue(dic)
travel.append(dic)
bigDic['travel']=travel
print bigDic

city   :   名古屋
planeCom   :   中華航空
title   :   【中華航空】漫步黑部立山大雪谷.白川鄉合掌村.惠那峽.飛驒牛溫泉４日
price   :   21900
day2   :   限定（04/16~06/22）『雪之大谷散策．零距離感受雪之迴廊』 *六種交通工具遊覽立山*扇澤～無軌電車～黑部水壩 → 黑部平～地下電纜車 → 大觀峰～鋼索纜車 → 室堂～隧道巴士 → 美女平～高原巴士 → 黑部阿爾卑斯山脈～立山連峰(中部山岳國家公園)～電纜車
day3   :   金澤兼六園(日本三大名園之一) →世界遺產～白川鄉合掌村(國家指定重要傳統建築物群保存區)→郡上八幡．宗祇水一郡上八幡舞蹈~加贈飛驒牛可樂餅+日式紅豆湯→免稅店～名古屋榮地下街或商店街逛街
day1   :   台北／名古屋空港 →惠那峽展望台～日本美景百選峽谷(欣賞國家天然保護物-傘岩與覆蓋著褐紅色苔的紅岩) →木曾古道紀行～妻籠宿（舊街）～感受江戶時代盛況
time   :   07:30 - 11:15
pid1   :   VDR0000001888
pid2   :   NGO04160602A
day4   :   名古屋→江戶風情商店街自由購物～中部空港／桃園國際機場
{'travel': [{'city': u'\u540d\u53e4\u5c4b', 'planeCom': u'\u4e2d\u83ef\u822a\u7a7a', 'title': u'\u3010\u4e2d\u83ef\u822a\u7a7a\u3011\u6f2b\u6b65\u9ed1\u90e8\u7acb\u5c71\u5927\u96ea\u8c37.\u767d\u5ddd\u9109\u5408\u638c\u6751.\u60e0\u90a3\u5cfd.\u98db\u9a52\u725b\u6eab\u6cc9\uff14\u65e5', 'price': u'21900', 'day2': u'\u9650\u5b9a\uff0804/16~06/22\uff09\u300e\u96ea\u4e4b\u5927\u8c37\u6563\u7b56\uff0e\u96f6\u8ddd\u96e2\u611f\u53d7\u96ea\u4e4b\u8ff4\u

<dictionary-itemiterator at 0x69af4f8>